# **Thermal Waves Extension**
This jupyter notebook contains the data analysis for the extension section of the script.


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from data_utils import load_dataset
from collections import defaultdict
import re
import os
from IPython.display import Math, display

## **Heat Diffusion through a Cylinder**

The flow of heat in a medium is described by the **heat equation**, which for a 1-D system) is:


$$
\frac{\partial T(x,t)}{\partial t} = D \frac{\partial^2 T(x,t)}{\partial x^2}
$$


where $T(x,t)$ is the temperature in the medium as a function of space and time, and $D$ is the **thermal diffusivity** of the medium.

---

### Analytical Solution
Using a trial solution of the form $T(x, t) = X(x)\,\theta(t)$ we can write a solution to this equation:

$$
T(x,t) = \left(C_{-}\,e^{\beta x} + C_{+}e^{-\beta x} \right)\,e^{-i\omega t}
$$
where $C_{-}$ and $C_{+}$ are complex amplitudes and $\beta$ is a complex propagation coefficient:
$$
\beta = (1+i)\sqrt{\frac{\omega}{2D}}
$$
We assume the top end of the cylinder ($x = L$) is non-conductive so:
$$
\left. \frac{\partial T}{\partial x} \right|_{x=L} = 0
$$

After applying this condition we can rewrite our solution in terms of a single complex amplitude $K$:

$$
T(x, t) = K \cosh\left( \beta(L-x) \right) \, e^{-i\omega t}
$$

---

### Geometry of the Experiment

In the plane slab model, the brass cylinder is treated as a **1-dimensional slab** of thickness

$$
x_i = i\,\Delta d
$$

with the x-axis along the cylinder’s axis.  

Thermal waves propagate along the x-axis, and the thermistor sensors are placed every $\Delta d = 5~\text{mm}$.

We define $x = 0$ at the first thermistor.


In [58]:
delta_d = 5  # 5 mm spacing between thermistors

In [59]:
save_dir = "Thermal Wave Data"  # The folder that contains the data

files = ['brass_T10s_run1.csv', 'brass_T10s_run2.csv', 'brass_T10s_run3.csv',
         'brass_T15s_run1.csv', 'brass_T15s_run2.csv', 'brass_T15s_run3.csv',
         'brass_T20s_run1.csv', 'brass_T20s_run2.csv', 'brass_T20s_run3.csv', 'brass_T20s_run4.csv',
         'brass_T30s_run1.csv', 'brass_T30s_run2.csv', 'brass_T30s_run3.csv',
         'brass_T45s_run1.csv', 'brass_T45s_run2.csv',
         'brass_T60s_run1.csv', 'brass_T60s_run2.csv', 'brass_T60s_run3.csv',
         'brass_T75s_run1.csv', 'brass_T75s_run2.csv',
         'brass_T90s_run1.csv', 'brass_T90s_run2.csv',
         'brass_T105s_run1.csv', 'brass_T105s_run2.csv', 'brass_T105s_run3.csv',
         'brass_T120s_run1.csv', 'brass_T120s_run2.csv', 'brass_T120s_run3.csv',
         'brass_T135s_run1.csv', 'brass_T135s_run2.csv', 'brass_T135s_run3.csv',
         'brass_T150s_run1.csv', 'brass_T150s_run2.csv', 'brass_T150s_run3.csv']

pattern = re.compile(r"T(\d+)s_run(\d+)")  # Name format to extract time period and run number

### **Manually Truncate Datasets to Remove Transients**

First we will plot all datasets to determine where to manually truncate them using a mask:

In [47]:
'''for file in files:
    path = os.path.join(save_dir, file)

    timestamp, V, I, Temperatures, comments = load_dataset(path)
    n_therm = Temperatures.shape[1]

    T, run = pattern.search(file).groups()

    plt.figure(figsize = (12, 5))

    for i in range(n_therm):
        plt.plot(timestamp, Temperatures[:, i], label = f"Thermistor {i}")

    plt.xlabel("Time [s]")
    plt.ylabel("Temperature [°C]")
    
    plt.title(f"Time Period = {T} s, Run {run}")

    plt.grid(True)
    plt.legend(ncol = 4, fontsize = 9)
    plt.tight_layout()
    plt.show()'''

'for file in files:\n    path = os.path.join(save_dir, file)\n\n    timestamp, V, I, Temperatures, comments = load_dataset(path)\n    n_therm = Temperatures.shape[1]\n\n    T, run = pattern.search(file).groups()\n\n    plt.figure(figsize = (12, 5))\n\n    for i in range(n_therm):\n        plt.plot(timestamp, Temperatures[:, i], label = f"Thermistor {i}")\n\n    plt.xlabel("Time [s]")\n    plt.ylabel("Temperature [°C]")\n    \n    plt.title(f"Time Period = {T} s, Run {run}")\n\n    plt.grid(True)\n    plt.legend(ncol = 4, fontsize = 9)\n    plt.tight_layout()\n    plt.show()'

In [48]:
# Truncation rules:
truncate_rules = {
        'brass_T10s_run1.csv': lambda t: t > 500,
        'brass_T10s_run2.csv': lambda t: t < 200,
        'brass_T10s_run3.csv': lambda t: t > 650,
        'brass_T15s_run1.csv': lambda t: t > 500,
        'brass_T15s_run2.csv': lambda t: t > 100,
        'brass_T15s_run3.csv': lambda t: t > 400,
        'brass_T20s_run1.csv': lambda t: t > 200,
        'brass_T20s_run2.csv': lambda t: t < 200,
        'brass_T20s_run3.csv': lambda t: t < 200,
        'brass_T20s_run4.csv': lambda t: t > 400,
        'brass_T30s_run1.csv': lambda t: t > 300,
        'brass_T30s_run2.csv': lambda t: t > 100,
        'brass_T30s_run3.csv': lambda t: t > 400,
        'brass_T45s_run1.csv': lambda t: t > 400,
        'brass_T45s_run2.csv': lambda t: t > 400,
        'brass_T60s_run1.csv': lambda t: t > 300,
        'brass_T60s_run2.csv': lambda t: t > 300,
        'brass_T60s_run3.csv': lambda t: t > 400,
        'brass_T75s_run1.csv': lambda t: t > 300,
        'brass_T75s_run2.csv': lambda t: t > 300,
        'brass_T90s_run1.csv': lambda t: t > 350,
        'brass_T90s_run2.csv': lambda t: t > 350,
        'brass_T105s_run1.csv': lambda t: t < 400,
        'brass_T105s_run2.csv': lambda t: t > 400,
        'brass_T105s_run3.csv': lambda t: t > 200,
        'brass_T120s_run1.csv': lambda t: t > 300,
        'brass_T120s_run2.csv': lambda t: t > 250,
        'brass_T120s_run3.csv': lambda t: t > 350,
        'brass_T135s_run1.csv': lambda t: t > 300,
        'brass_T135s_run2.csv': lambda t: t > 300,
        'brass_T135s_run3.csv': lambda t: t > 250,
        'brass_T150s_run1.csv': lambda t: t > 250,
        'brass_T150s_run2.csv': lambda t: t > 350,
        'brass_T150s_run3.csv': lambda t: t > 350
}

### **Fit Each Thermistor to a Sinusoid**

For each time period $\tau$ we will fit each thermistor to a sinusoid of the form:

$$
T(t) = T_0 + A\sin\left( \omega t + \phi \right), \quad \omega = \frac{2\pi}{\tau}
$$

to extract their amplitudes and phases.


In [49]:
def sinusoid(t, T0, A, phi, omega):
    return T0 + A * np.sin(omega * t + phi)

In [50]:
def fit_thermistor(time, temp, period):
    
    omega = 2 * np.pi / period

    # Initial guesses
    T0_guess = np.mean(temp)
    A_guess = 0.5 * (np.max(temp) - np.min(temp))
    phi_guess = 0.0

    popt, pcov = curve_fit(
        lambda t, T0, A, phi: sinusoid(t, T0, A, phi, omega),
        time,
        temp,
        p0 = [T0_guess, A_guess, phi_guess]
    )
    T0, A, phi = popt
    
    # Enforce A ≥ 0
    if A < 0:
        A = -A
        phi = phi + np.pi
        
    return T0, A, phi, pcov

### **Compute Complex Amplitude**

Using our extracted amplitudes and phases we define a complex amplitude:

$$
\tilde T = Ae^{i\phi}
$$

A single complex number now contains both the phase and amplitude information. We will now define a function to process each file, returning a dictionary of the complex amplitudes, their uncertainties and other metadata for each thermistor:

In [51]:
def process_file(file):

    path = os.path.join(save_dir, file)
    timestamp, V, I, Temps, comments = load_dataset(path)

    # Extract time period and run number from filename
    T_str, run_str = pattern.search(file).groups()
    period = float(T_str)

    # Apply truncation mask to remove transients
    mask = truncate_rules[file](timestamp)
    t_fit = timestamp[mask]
    temps_fit = Temps[mask, :]

    n_therm = temps_fit.shape[1]

    # Fit all thermistors
    T0_list = []
    A_list = []
    phi_list = []
    cov_list = []

    for i in range(n_therm):
        T0_i, A_i, phi_i, pcov_i = fit_thermistor(t_fit, temps_fit[:, i], period)
        T0_list.append(T0_i)
        A_list.append(A_i)
        phi_list.append(phi_i)
        cov_list.append(pcov_i)

    T0_arr = np.array(T0_list)
    A_arr = np.array(A_list)
    phi_arr = np.array(phi_list)

    # Complex amplitudes
    T_complex = A_arr * np.exp(1j * phi_arr)

    return period, {
        "file": file,
        "run": int(run_str),
        "t_fit": t_fit,
        "temps_fit": temps_fit,
        "T0": T0_arr,
        "A": A_arr,
        "phi": phi_arr,
        "cov": cov_list,
        "T_complex": T_complex
    }

In [52]:
raw_results_by_period = defaultdict(list)

for file in files:
    period, run_result = process_file(file)
    raw_results_by_period[period].append(run_result)

### **Compute Dimentionless Amplitude Ratio**

For each thermistor $i = 0, 1, 2, \dots , 7$ we normalise each run by the complex amplitude of thermistor 0 to form the dimensionless ratio:

$$
R_i = \frac{\tilde T_i}{\tilde T_0} = \frac{A_i}{A_0}e^{i(\phi_i - \phi_0)}.
$$

We then average this ratio across the runs for each time period to obtain an estimate of the statistical uncertainty:


In [53]:
summary_by_period = {}

for period, run_list in raw_results_by_period.items():
    # Stack complex amplitudes: shape (n_runs, n_therm)
    T_complex_runs = np.vstack([r["T_complex"] for r in run_list])
    n_runs, n_therm = T_complex_runs.shape

    # Compute ratios for each run: R_i^(r) = T_i^(r) / T_0^(r)
    R_runs = T_complex_runs / T_complex_runs[:, [0]]  # normalise each row by its first element

    # Mean and std across runs
    T_complex_mean = np.mean(T_complex_runs, axis=0)
    T_complex_std = np.std(T_complex_runs, axis=0, ddof=1)

    R_mean = np.mean(R_runs, axis=0)
    R_std = np.std(R_runs, axis=0, ddof=1)

    summary_by_period[period] = {
        "n_runs": n_runs,
        "T_complex_runs": T_complex_runs,
        "T_complex_mean": T_complex_mean,
        "T_complex_std": T_complex_std,
        "R_runs": R_runs,
        "R_mean": R_mean,
        "R_std": R_std,
    }

### **Inspect R_mean and R_std for each period**

for period in sorted(summary_by_period.keys()):
    R_mean = summary_by_period[period]["R_mean"]
    R_std = summary_by_period[period]["R_std"]
    n_runs = summary_by_period[period]["n_runs"]

    print(f"\n==============================")
    print(f"      Period T = {period:.0f} s")
    print(f"      Runs: {n_runs}")
    print("==============================\n")

    print("Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg(R_mean) [rad]      |R_std|")
    print("-------------------------------------------------------------------------------------------")

    for i, R in enumerate(R_mean):
        R_abs = np.abs(R)
        R_phase = np.angle(R)
        R_err = np.abs(R_std[i])  # magnitude of complex std

        print(f"{i:5d} | {R.real: .4e}   {R.imag: .4e}   {R_abs: .4e}        {R_phase: .4f}           {R_err: .4e}")



      Period T = 10 s
      Runs: 3

Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg(R_mean) [rad]      |R_std|
-------------------------------------------------------------------------------------------
    0 |  1.0000e+00    1.8487e-19    1.0000e+00         0.0000            1.5370e-17
    1 |  5.2797e-01   -2.9414e-01    6.0437e-01        -0.5083            3.3158e-03
    2 |  1.8012e-01   -3.1505e-01    3.6290e-01        -1.0514            8.9727e-04
    3 |  2.5840e-02   -2.3226e-01    2.3369e-01        -1.4600            9.5805e-04
    4 | -5.3771e-02   -1.2452e-01    1.3563e-01        -1.9784            1.1986e-03
    5 | -6.6058e-02   -4.7147e-02    8.1157e-02        -2.5217            1.3804e-03
    6 | -5.4424e-02   -7.9266e-04    5.4429e-02        -3.1270            1.4100e-03
    7 | -4.3092e-02    2.3116e-02    4.8901e-02         2.6492            1.7398e-03

      Period T = 15 s
      Runs: 3

Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg

In [54]:
### Theoretical models for complex ratio R_i = T̃_i / T̃_0

def R_exp_model(D, x, omega):
    """Exponential thermal-wave model (one-way propagation)."""
    beta = (1 + 1j) * np.sqrt(omega / (2 * D))
    return np.exp(-beta * (x - x[0]))


def R_cosh_model(D, x, omega, L):
    """Full cosh model including reflection at non-conductive end."""
    beta = (1 + 1j) * np.sqrt(omega / (2 * D))
    # Normalised ratio:
    return np.cosh(beta * (L - x)) / np.cosh(beta * (L - x[0]))


In [55]:
from scipy.optimize import least_squares

def fit_model(R_mean, R_runs, x, omega, model, L=None):
    """
    Fit either the exponential ('exp') or cosh ('cosh') model
    to complex ratio data R_mean, using run-to-run scatter in
    R_runs to estimate uncertainties.
    """
    R_mean = np.asarray(R_mean)
    R_runs = np.asarray(R_runs)

    n_therm = len(x)
    fit_idx = np.arange(1, n_therm)  # skip thermistor 0 (R = 1 by definition)

    # --- empirical uncertainties from run-to-run scatter ---
    sigma_real = np.std(R_runs.real, axis=0, ddof=1)
    sigma_imag = np.std(R_runs.imag, axis=0, ddof=1)

    # ensure they're at least 1D arrays
    sigma_real = np.atleast_1d(sigma_real)
    sigma_imag = np.atleast_1d(sigma_imag)

    # avoid zero/tiny sigma → prevents division by zero or huge weights
    min_sigma = 1e-6
    sigma_real = np.maximum(sigma_real, min_sigma)
    sigma_imag = np.maximum(sigma_imag, min_sigma)

    # --- choose model ---
    def model_func(D):
        if model == "exp":
            return R_exp_model(D, x, omega)
        elif model == "cosh":
            return R_cosh_model(D, x, omega, L)
        else:
            raise ValueError("Unknown model type")

    # --- residuals: real and imag parts weighted separately ---
    def residuals(logD):
        # handle scalar or array
        logD = np.atleast_1d(logD)
        D = np.exp(logD[0])  # D > 0

        R_model = model_func(D)
        r = R_mean - R_model

        res_real = (r.real / sigma_real)[fit_idx]
        res_imag = (r.imag / sigma_imag)[fit_idx]

        return np.concatenate([res_real, res_imag])

    # initial guess
    logD0 = np.log(1e-5)
    result = least_squares(residuals, x0=[logD0])

    D_fit = np.exp(result.x[0])
    res_final = result.fun              # residuals at best fit (already computed)
    chi2 = np.sum(res_final**2)

    n_params = 1
    n_data = res_final.size

    AIC = chi2 + 2 * n_params
    BIC = chi2 + n_params * np.log(n_data)

    return {
        "D_fit": D_fit,
        "chi2": chi2,
        "AIC": AIC,
        "BIC": BIC,
        "residuals": res_final,
        "result": result,
    }


In [56]:
L_mm = 41.0
L_m  = L_mm / 1000.0

x_mm = np.arange(8) * delta_d
x_m  = x_mm / 1000.0


In [57]:
fit_results = {}

for period, data in sorted(summary_by_period.items()):
    omega = 2 * np.pi / period

    R_mean = data["R_mean"]
    R_runs = data["R_runs"]

    fit_exp  = fit_model(R_mean, R_runs, x_m, omega, model="exp",  L=L_m)
    fit_cosh = fit_model(R_mean, R_runs, x_m, omega, model="cosh", L=L_m)

    fit_results[period] = {
        "exp":  fit_exp,
        "cosh": fit_cosh,
    }

    print(f"\n======================")
    print(f"  T = {period:.0f} s fits")
    print("======================")
    print(f"Exponential:  D = {fit_exp['D_fit']:.3e},  χ² = {fit_exp['chi2']:.3f}, "
          f"AIC = {fit_exp['AIC']:.2f}, BIC = {fit_exp['BIC']:.2f}")
    print(f"Cosh:         D = {fit_cosh['D_fit']:.3e},  χ² = {fit_cosh['chi2']:.3f}, "
          f"AIC = {fit_cosh['AIC']:.2f}, BIC = {fit_cosh['BIC']:.2f}")



  T = 10 s fits
Exponential:  D = 3.054e-05,  χ² = 2388.156, AIC = 2390.16, BIC = 2390.80
Cosh:         D = 3.052e-05,  χ² = 2295.665, AIC = 2297.66, BIC = 2298.30

  T = 15 s fits
Exponential:  D = 3.092e-05,  χ² = 891.836, AIC = 893.84, BIC = 894.48
Cosh:         D = 3.261e-05,  χ² = 530.273, AIC = 532.27, BIC = 532.91

  T = 20 s fits
Exponential:  D = 3.051e-05,  χ² = 1956.101, AIC = 1958.10, BIC = 1958.74
Cosh:         D = 3.091e-05,  χ² = 1128.487, AIC = 1130.49, BIC = 1131.13

  T = 30 s fits
Exponential:  D = 3.595e-05,  χ² = 9149.823, AIC = 9151.82, BIC = 9152.46
Cosh:         D = 3.595e-05,  χ² = 1485.440, AIC = 1487.44, BIC = 1488.08

  T = 45 s fits
Exponential:  D = 2.737e-05,  χ² = 130610.185, AIC = 130612.18, BIC = 130612.82
Cosh:         D = 3.280e-05,  χ² = 41905.856, AIC = 41907.86, BIC = 41908.50

  T = 60 s fits
Exponential:  D = 2.881e-05,  χ² = 25178.433, AIC = 25180.43, BIC = 25181.07
Cosh:         D = 3.514e-05,  χ² = 1964.565, AIC = 1966.56, BIC = 1967.20

  T

In [60]:
### Theoretical models for complex ratio R_i = T̃_i / T̃_0
### All distances in mm, D in mm^2/s, ω in rad/s.

def R_exp_model(D_mm2_s, x_mm, omega):
    """
    Semi-infinite exponential (plane-wave) model:

        T̃(x) ∝ exp[-(1+i) α x],   α = sqrt(ω / (2D)),

    so for thermistor at x_i relative to x_0:

        R_i = T̃(x_i) / T̃(x_0)
            = exp[-(1+i) sqrt(ω/(2D)) (x_i - x_0)].
    """
    beta = (1.0 + 1.0j) * np.sqrt(omega / (2.0 * D_mm2_s))  # 1/mm
    return np.exp(-beta * (x_mm - x_mm[0]))


def R_cosh_model(D_mm2_s, x_mm, omega, L_mm):
    """
    Finite-length cosh model with insulated end at x = L:

        T̃(x) ∝ cosh[β (L - x)],   β = (1+i) sqrt(ω/(2D)),

    so the normalised complex ratio is

        R_i = T̃(x_i) / T̃(x_0)
            = cosh[β (L - x_i)] / cosh[β (L - x_0)].
    """
    beta = (1.0 + 1.0j) * np.sqrt(omega / (2.0 * D_mm2_s))  # 1/mm

    num   = np.cosh(beta * (L_mm - x_mm))
    denom = np.cosh(beta * (L_mm - x_mm[0]))  # = cosh(β L_mm) if x_0 = 0

    return num / denom


In [61]:
from scipy.optimize import least_squares

def fit_model(R_mean, R_runs, x_mm, omega, model, L_mm=None, fit_max_index=4):
    """
    Fit either the exponential ('exp') or cosh ('cosh') model
    to complex ratio data R_mean, using run-to-run scatter in
    R_runs to estimate uncertainties.

    Parameters
    ----------
    R_mean : array (n_therm,)
        Mean complex ratio R_i = T̃_i / T̃_0 over runs.
    R_runs : array (n_runs, n_therm)
        Complex ratios for each run.
    x_mm : array (n_therm,)
        Thermistor positions in mm.
    omega : float
        Angular frequency (rad/s).
    model : 'exp' or 'cosh'
    L_mm : float
        Cylinder length in mm (needed for cosh model).
    fit_max_index : int
        Last thermistor index to include in the fit (we fit i = 1..fit_max_index).

    Returns
    -------
    dict with keys 'D_fit', 'chi2', 'AIC', 'BIC', 'residuals', 'result'
    """
    R_mean = np.asarray(R_mean)   # shape (n_therm,)
    R_runs = np.asarray(R_runs)   # shape (n_runs, n_therm)
    n_runs, n_therm = R_runs.shape

    # Use thermistors 1..fit_max_index (skip 0 because R_0 = 1 by definition)
    fit_max_index = min(fit_max_index, n_therm - 1)
    fit_idx = np.arange(1, fit_max_index + 1)

    # --- empirical uncertainties from run-to-run scatter ---
    if n_runs > 1:
        sigma_real = np.std(R_runs.real, axis=0, ddof=1)
        sigma_imag = np.std(R_runs.imag, axis=0, ddof=1)
    else:
        # If only one run, assume a fixed fractional uncertainty (~5%)
        frac = 0.05
        sigma_real = frac * np.maximum(np.abs(R_mean.real), 1e-3)
        sigma_imag = frac * np.maximum(np.abs(R_mean.imag), 1e-3)

    min_sigma = 1e-3
    # Replace NaNs/Infs and enforce a minimum
    sigma_real = np.where(np.isfinite(sigma_real), sigma_real, min_sigma)
    sigma_imag = np.where(np.isfinite(sigma_imag), sigma_imag, min_sigma)
    sigma_real = np.maximum(sigma_real, min_sigma)
    sigma_imag = np.maximum(sigma_imag, min_sigma)

    # --- choose model ---
    def model_func(D):
        if model == "exp":
            return R_exp_model(D, x_mm, omega)
        elif model == "cosh":
            return R_cosh_model(D, x_mm, omega, L_mm)
        else:
            raise ValueError("Unknown model type")

    # --- residuals: real and imag parts weighted separately ---
    def residuals(logD):
        # logD is scalar -> D > 0
        D = np.exp(logD[0])

        R_model = model_func(D)
        r = R_mean - R_model

        res_real = (r.real / sigma_real)[fit_idx]
        res_imag = (r.imag / sigma_imag)[fit_idx]

        return np.concatenate([res_real, res_imag])

    # initial guess for D ~ 35 mm^2/s (brass-ish)
    logD0 = np.log(35.0)

    result = least_squares(residuals, x0=[logD0])

    D_fit = float(np.exp(result.x[0]))
    res_final = result.fun
    chi2 = float(np.sum(res_final**2))

    n_params = 1
    n_data = res_final.size

    AIC = chi2 + 2 * n_params
    BIC = chi2 + n_params * np.log(n_data)

    return {
        "D_fit": D_fit,
        "chi2": chi2,
        "AIC": AIC,
        "BIC": BIC,
        "residuals": res_final,
        "result": result,
    }


In [62]:
L_mm = 41.0                      # cylinder length in mm
# Infer number of thermistors from any period
any_period = next(iter(summary_by_period.keys()))
n_therm = summary_by_period[any_period]["R_mean"].shape[0]
x_mm = np.arange(n_therm) * delta_d   # positions of thermistors in mm

fit_results = {}

for period, data in sorted(summary_by_period.items()):
    omega = 2 * np.pi / period

    R_mean = data["R_mean"]   # shape (n_therm,)
    R_runs = data["R_runs"]   # shape (n_runs, n_therm)

    fit_exp  = fit_model(R_mean, R_runs, x_mm, omega, model="exp",  L_mm=L_mm)
    fit_cosh = fit_model(R_mean, R_runs, x_mm, omega, model="cosh", L_mm=L_mm)

    fit_results[period] = {
        "exp":  fit_exp,
        "cosh": fit_cosh,
    }

    print(f"\n======================")
    print(f"  T = {period:.0f} s fits")
    print("======================")
    print(f"Exponential:  D = {fit_exp['D_fit']:.3f} mm^2/s,  "
          f"χ² = {fit_exp['chi2']:.3f}, AIC = {fit_exp['AIC']:.2f}, BIC = {fit_exp['BIC']:.2f}")
    print(f"Cosh:         D = {fit_cosh['D_fit']:.3f} mm^2/s,  "
          f"χ² = {fit_cosh['chi2']:.3f}, AIC = {fit_cosh['AIC']:.2f}, BIC = {fit_cosh['BIC']:.2f}")



  T = 10 s fits
Exponential:  D = 31.249 mm^2/s,  χ² = 583.370, AIC = 585.37, BIC = 585.45
Cosh:         D = 31.136 mm^2/s,  χ² = 550.955, AIC = 552.95, BIC = 553.03

  T = 15 s fits
Exponential:  D = 31.139 mm^2/s,  χ² = 107.760, AIC = 109.76, BIC = 109.84
Cosh:         D = 31.550 mm^2/s,  χ² = 117.551, AIC = 119.55, BIC = 119.63

  T = 20 s fits
Exponential:  D = 30.380 mm^2/s,  χ² = 229.202, AIC = 231.20, BIC = 231.28
Cosh:         D = 30.418 mm^2/s,  χ² = 237.413, AIC = 239.41, BIC = 239.49

  T = 30 s fits
Exponential:  D = 29.080 mm^2/s,  χ² = 264.594, AIC = 266.59, BIC = 266.67
Cosh:         D = 31.465 mm^2/s,  χ² = 464.167, AIC = 466.17, BIC = 466.25

  T = 45 s fits
Exponential:  D = 26.921 mm^2/s,  χ² = 5555.788, AIC = 5557.79, BIC = 5557.87
Cosh:         D = 33.081 mm^2/s,  χ² = 2340.422, AIC = 2342.42, BIC = 2342.50

  T = 60 s fits
Exponential:  D = 26.718 mm^2/s,  χ² = 4354.544, AIC = 4356.54, BIC = 4356.62
Cosh:         D = 33.603 mm^2/s,  χ² = 946.480, AIC = 948.48, BI